In [1]:
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet34
from torchvision import transforms
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier, VotingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import random
import joblib

# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Preparation

In [3]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/attack/pickle/cifar10/resnet34/shadow.p'

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)

print(type(dataset), len(dataset))

# split the dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

cuda:0
<class 'list'> 30000


# shadow model

In [ ]:
# model parameters
TARGET_MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/attack/models/resnet34_cifar10.pth'
n_classes = 10
n_epochs = 50
batch_size = 64
learning_rate = 0.001
weight_decay = 1e-4  # parameter for L2 regularization

# load target model for training shadow model
shadow_model = resnet34(num_classes=n_classes).to(device)
state_dict = torch.load(TARGET_MODEL_PATH, map_location=device)
shadow_model.load_state_dict(state_dict['net'])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(shadow_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# model training
shadow_model.train()
for epoch in range(n_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = shadow_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()

    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {running_loss/len(train_loader)}")
    # save the model after each 5 epochs
    if (epoch+1) % 5 == 0:
        torch.save(shadow_model.state_dict(), f"/content/drive/My Drive/Colab Notebooks/attack/my_models/cifar10_resnet/shadow_model_epoch_{epoch+1}.pt")

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/50, Loss: 0.953537145614624
Epoch 2/50, Loss: 0.7045986836751302
Epoch 3/50, Loss: 0.5461268642346064
Epoch 4/50, Loss: 0.4314792303244273
Epoch 5/50, Loss: 0.33479435177644096
Epoch 6/50, Loss: 0.25611126454671224
Epoch 7/50, Loss: 0.21289947843551635
Epoch 8/50, Loss: 0.17704525246222813
Epoch 9/50, Loss: 0.16731408825019994
Epoch 10/50, Loss: 0.148850849678119
Epoch 11/50, Loss: 0.063306514116625
Epoch 12/50, Loss: 0.025057315078874428
Epoch 13/50, Loss: 0.015103649364163477
Epoch 14/50, Loss: 0.010856496345562239
Epoch 15/50, Loss: 0.007141458673713108
Epoch 16/50, Loss: 0.0059167006482991075
Epoch 17/50, Loss: 0.004763339297535519
Epoch 18/50, Loss: 0.0035495820069530357
Epoch 19/50, Loss: 0.003403091239432494
Epoch 20/50, Loss: 0.003522117187657083
Epoch 21/50, Loss: 0.002548049951554276
Epoch 22/50, Loss: 0.0021525892882297438
Epoch 23/50, Loss: 0.00190366480499506
Epoch 24/50, Loss: 0.0015825337367520358
Epoch 25/50, Loss: 0.0017289480246060218
Epoch 26/50, Loss: 0.0016

In [5]:
# load trained shadow model
n_classes = 10
criterion = nn.CrossEntropyLoss()

shadow_model = resnet34(num_classes=n_classes).to(device)
shadow_model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/attack/my_models/cifar10_resnet/shadow_model_epoch_50.pt"))

<All keys matched successfully>

In [6]:
# model evaluation
test_loss = 0.0
correct = 0
total = 0
shadow_model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = shadow_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1) # get the class index
        total += labels.size(0) # total samples
        correct += (predicted == labels).sum().item() # correctly predicted samples

print(f'Test Loss: {test_loss/len(test_loader)}, Accuracy: {100 * correct / total}%')

Test Loss: 1.4883059028615342, Accuracy: 73.8%


# Attack Model Training

In [25]:
# method for generating attack dataset for given model and dataset
def generate_attack_data(model, data_loader, label):
    attack_data = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = nn.functional.softmax(outputs, dim=1)
            for prob in probabilities:
                attack_data.append((prob.cpu().numpy(), label))
    return attack_data

# generate attack data
member_data = generate_attack_data(shadow_model, train_loader, 1) # member label as 1
non_member_data = generate_attack_data(shadow_model, test_loader, 0) # non-member label as 0
attack_data = member_data + non_member_data

# prepare the data for training attack model
np.random.shuffle(attack_data)
X = np.array([x[0] for x in attack_data])
y = np.array([x[1] for x in attack_data])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [27]:
# train attack model
attack_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=12,
    learning_rate=0.03,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    subsample=0.9,
    colsample_bytree=0.5,
    reg_alpha=0.1,
    reg_lambda=4.0,
    early_stopping_rounds=30,
    scale_pos_weight=0.2,
    tree_method = 'hist',
    device = 'gpu'
)

attack_model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_test, y_test)],
                 verbose=True)

y_pred = attack_model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f"Attack Model Accuracy on train set: {accuracy}")

# evaluate the attack model on the test data
y_pred = attack_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Attack Model Accuracy on test set: {accuracy}")

[0]	validation_0-logloss:0.76240	validation_1-logloss:0.76629
[1]	validation_0-logloss:0.75401	validation_1-logloss:0.75860
[2]	validation_0-logloss:0.74580	validation_1-logloss:0.75121
[3]	validation_0-logloss:0.73768	validation_1-logloss:0.74371
[4]	validation_0-logloss:0.73025	validation_1-logloss:0.73708
[5]	validation_0-logloss:0.72317	validation_1-logloss:0.73071
[6]	validation_0-logloss:0.71590	validation_1-logloss:0.72407
[7]	validation_0-logloss:0.70919	validation_1-logloss:0.71828
[8]	validation_0-logloss:0.70277	validation_1-logloss:0.71243
[9]	validation_0-logloss:0.69641	validation_1-logloss:0.70675
[10]	validation_0-logloss:0.69049	validation_1-logloss:0.70165
[11]	validation_0-logloss:0.68436	validation_1-logloss:0.69633
[12]	validation_0-logloss:0.67861	validation_1-logloss:0.69123
[13]	validation_0-logloss:0.67289	validation_1-logloss:0.68626
[14]	validation_0-logloss:0.66704	validation_1-logloss:0.68111
[15]	validation_0-logloss:0.66194	validation_1-logloss:0.67663
[1

# Target Model

In [28]:
# load the target model
MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/attack/models/resnet34_cifar10.pth'

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = resnet34(num_classes=10).to(device)

state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

print(target_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [29]:
# loading evaluation data
with open('/content/drive/My Drive/Colab Notebooks/attack/pickle/cifar10/resnet34/eval.p', 'rb') as f:
    eval_dataset = pickle.load(f)

# prepare the data for the evaluation
eval_images = [item[0] for item in eval_dataset]
eval_labels = [item[1] for item in eval_dataset]
eval_membership_status = [item[2] for item in eval_dataset]

eval_images_tensor = torch.stack(eval_images)
eval_labels_tensor = torch.tensor(eval_labels)
eval_dataset_tensor = TensorDataset(eval_images_tensor, eval_labels_tensor)
eval_loader = DataLoader(eval_dataset_tensor, batch_size=64, shuffle=False, num_workers=2)


In [31]:
#  Generate evaluation data for attack model using target model
probabilities = []
target_model.eval()
with torch.no_grad():
    for inputs, _ in eval_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())

X_eval = np.array(probabilities)
y_eval_true = np.array(eval_membership_status)


In [32]:
# evaluate the attack model
y_eval_pred = attack_model.predict(X_eval)
accuracy = accuracy_score(y_eval_true, y_eval_pred)
print(f"Membership Inference Accuracy on evaluation data: {accuracy}")

Membership Inference Accuracy on evaluation data: 0.745


In [33]:
joblib.dump(attack_model, '/content/drive/My Drive/Colab Notebooks/attack/attack_models/cifar10_resnet/attack_model_74.pkl')

['/content/drive/My Drive/Colab Notebooks/attack/attack_models/cifar10_resnet/attack_model_74.pkl']